## Natural Language Processing
### Using the Bag of Words algorithm

In [118]:
# Load in the data THIS DATASET IS TAB SEPERATED
import os
import pandas as pd

DATASET_PATH = "datasets/"
DATASET_NAME = "Restaurant_Reviews.tsv"
DATASET_URL = DATASET_PATH + DATASET_NAME

def fetch_data(dataset_url=DATASET_URL, dataset_path=DATASET_PATH):
    if not os.path.isdir(dataset_path):
        os.makedirs(dataset_path)

dataset = fetch_data()

def load_data(dataset_path=DATASET_PATH, dataset_name=DATASET_NAME):
    csv_path = os.path.join(dataset_path, dataset_name)
    return pd.read_csv(csv_path, delimiter='\t', quoting = 3)

dataset = load_data()

In [3]:
dataset.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [32]:
index = dataset['Review'].shape
index = index[0]
print(index)

1000


In [119]:
# Any natural language processing algorithm needs to clean the dataset
import re
import nltk

nltk.download('stopwords') #stop words helps remove words that aren't considered 'relevant'
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

clean_dataset = []

for i in range(0, index):
    review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = " ".join(review)
    clean_dataset.append(review)
print(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/werlingk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
wast enough life pour salt wound draw time took bring check


In [120]:
# Build out a big ol' bag of words model
#this tool can be used as a shortcut but I broke everything out above should I need to customize it later
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(clean_dataset).toarray()

In [37]:
X.shape

(1000, 1500)

In [121]:
y = dataset['Liked'].values
y.shape

(1000,)

In [122]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

In [123]:
# Create Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB
nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)

y_pred = nb_clf.predict(X_test)

In [124]:
# Check out the performance
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

cf_mat= confusion_matrix(y_test, y_pred)
tp = cf_mat[0,0]
tn = cf_mat[1,1]
fp = cf_mat[1,0]
fn = cf_mat[0,1]
total = tp+tn+fp+fn

print("Precision: %.2f" % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))
print('F1 Score: %.2f' % f1_score(y_test, y_pred))
print("Percentage True Positive: %.2f" % ((tp/total)*100))
print("Percentage True Negative: %.2f" % ((tn/total)*100))
confusion_matrix(y_test, y_pred)

Precision: 0.76
Recall: 0.79
F1 Score: 0.78
Percentage True Positive: 36.00
Percentage True Negative: 40.50


array([[72, 25],
       [22, 81]])

In [125]:
acc_clf = round(nb_clf.score(X_train, y_train) * 100, 2)
print("Training set performance: %.2f" % acc_clf)

Training set performance: 93.88


In [126]:
acc_clf = round(nb_clf.score(X_test, y_test) * 100, 2)
print("Test set performance: %.2f" % acc_clf)

Test set performance: 76.50


In [96]:
# Try SVM
from sklearn.linear_model import SGDClassifier
sv_clf = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)
sv_clf.fit(X_train, y_train)

y_pred = sv_clf.predict(X_test)

In [97]:
# Check out the performance
cf_mat= confusion_matrix(y_test, y_pred)
tp = cf_mat[0,0]
tn = cf_mat[1,1]
fp = cf_mat[1,0]
fn = cf_mat[0,1]
total = tp+tn+fp+fn

print("Precision: %.2f" % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))
print('F1 Score: %.2f' % f1_score(y_test, y_pred))
print("Percentage True Positive: %.2f" % ((tp/total)*100))
print("Percentage True Negative: %.2f" % ((tn/total)*100))
confusion_matrix(y_test, y_pred)

Precision: 0.70
Recall: 0.81
F1 Score: 0.75
Percentage True Positive: 30.50
Percentage True Negative: 41.50


array([[61, 36],
       [20, 83]])

In [98]:
acc_clf = round(sv_clf.score(X_train, y_train) * 100, 2)
print("Training set performance: %.2f" % acc_clf)

Training set performance: 94.25


In [99]:
acc_clf = round(sv_clf.score(X_test, y_test) * 100, 2)
print("Test set performance: %.2f" % acc_clf)

Test set performance: 72.00


In [87]:
# Create Decision Tree Classifier
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(criterion='entropy', random_state = 0, n_estimators=12)
rf_clf.fit(X_train, y_train)

y_pred = rf_clf.predict(X_test)

In [88]:
# Check out the performance
cf_mat= confusion_matrix(y_test, y_pred)
tp = cf_mat[0,0]
tn = cf_mat[1,1]
fp = cf_mat[1,0]
fn = cf_mat[0,1]
total = tp+tn+fp+fn

print("Precision: %.2f" % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))
print('F1 Score: %.2f' % f1_score(y_test, y_pred))
print("Percentage True Positive: %.2f" % ((tp/total)*100))
print("Percentage True Negative: %.2f" % ((tn/total)*100))
confusion_matrix(y_test, y_pred)

Precision: 0.83
Recall: 0.57
F1 Score: 0.68
Percentage True Positive: 42.50
Percentage True Negative: 29.50


array([[85, 12],
       [44, 59]])

In [89]:
acc_clf = round(rf_clf.score(X_train, y_train) * 100, 2)
print("Training set performance: %.2f" % acc_clf)

Training set performance: 98.25


In [90]:
acc_clf = round(rf_clf.score(X_test, y_test) * 100, 2)
print("Test set performance: %.2f" % acc_clf)

Test set performance: 72.00


In [130]:
# Lets try a grid search for Naive Bayes since it was the most promising
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()), ('clf', MultinomialNB()) ])

parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3) }

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(X_train, y_train)
y_pred = gs_clf.predict(X_test)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/local/anaconda3/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/usr/local/anaconda3/lib/python